# Aegis Eval Quickstart

This notebook demonstrates how to run an evaluation with Aegis, inspect results,
compare runs, and generate reports.

**Prerequisites:** Install aegis with `pip install -e '.[dev]'` from the repo root.

In [ ]:
from aegis import EvalConfig, Evaluator

## 1. Run a Full Evaluation

The `Evaluator` runs cases across all 101 dimensions by default.
Each dimension gets a score between 0 and 1.

In [ ]:
evaluator = Evaluator(config=EvalConfig(dimensions="all"))
result = evaluator.run()

print(f"Overall score: {result.overall_score:.2%}")
print(f"Run ID:        {result.run_id}")
print(f"Dimensions:    {len(result.dimension_scores)}")

## 2. Inspect Dimension Scores

Scores are stored in `result.dimension_scores` as a dict mapping dimension ID to score.

In [ ]:
sorted_dims = sorted(result.dimension_scores.items(), key=lambda kv: kv[1], reverse=True)

print("Top 10 dimensions:")
for dim_id, score in sorted_dims[:10]:
    bar = '\u2588' * int(score * 40)
    print(f"  {dim_id:<40s} {score:.4f} {bar}")

print("\nBottom 10 dimensions:")
for dim_id, score in sorted_dims[-10:]:
    bar = '\u2588' * int(score * 40)
    print(f"  {dim_id:<40s} {score:.4f} {bar}")

## 3. Scores by Tier

Aegis organises dimensions into 7 tiers. Let's compute mean scores per tier.

In [ ]:
from collections import defaultdict

from aegis.eval.dimensions.registry import DimensionRegistry

registry = DimensionRegistry.instance()
tier_scores = defaultdict(list)

for dim_id, score in result.dimension_scores.items():
    dim = registry.get(dim_id)
    if dim:
        tier_scores[dim.tier].append(score)

print("Mean score by tier:")
for tier in sorted(tier_scores):
    scores = tier_scores[tier]
    mean = sum(scores) / len(scores)
    print(f"  Tier {tier}: {mean:.4f}  ({len(scores)} dimensions)")

## 4. Custom Evaluation

Select specific dimensions, set a seed for reproducibility, and adjust difficulty.

In [ ]:
config = EvalConfig(
    dimensions=[
        "retention_accuracy",
        "temporal_ordering",
        "contradiction_detection",
        "hallucination_rate",
    ],
    seed=42,
    difficulty="medium",
)

custom_result = Evaluator(config=config).run()

print(f"Overall score: {custom_result.overall_score:.2%}")
for dim_id, score in custom_result.dimension_scores.items():
    print(f"  {dim_id}: {score:.4f}")

## 5. Compare Two Runs

Run two evaluations with different seeds and compare dimension-level deltas.

In [ ]:
evaluator = Evaluator()

result_a = evaluator.run(config=EvalConfig(dimensions="all", seed=1))
result_b = evaluator.run(config=EvalConfig(dimensions="all", seed=2))

comparison = evaluator.compare(result_a, result_b)

print(f"Overall delta: {comparison['overall_delta']:+.4f}")
print(f"Improved:  {len(comparison['improved'])} dimensions")
print(f"Regressed: {len(comparison['regressed'])} dimensions")
print(f"Unchanged: {len(comparison['unchanged'])} dimensions")

deltas = sorted(comparison["dimension_deltas"].items(), key=lambda kv: abs(kv[1]), reverse=True)
print("\nBiggest changes:")
for dim_id, delta in deltas[:5]:
    direction = "+" if delta > 0 else ""
    print(f"  {dim_id:<40s} {direction}{delta:.4f}")

## 6. Export Results as JSON

All result objects are Pydantic models and can be serialised to JSON.

In [ ]:
import json

report = json.loads(result.model_dump_json())
print(json.dumps({
    "run_id": report["run_id"],
    "overall_score": report["overall_score"],
    "num_dimensions": len(report["dimension_scores"]),
}, indent=2))

## Next Steps

- **Domain evals**: See `02_domain_eval.ipynb` for legal & finance benchmark suites
- **Memory**: See `03_memory_operations.ipynb` for AegisMemory API
- **Ingestion**: See `04_ingestion_pipeline.ipynb` for document processing
- **CLI**: Run `aegis eval run --config eval.yaml` for headless evaluation
- **API**: POST `/api/v1/eval/run` for programmatic access